# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании. Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics import r2_score

In [2]:
data=pd.read_csv('/datasets/insurance.csv') 
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.isna().sum() 

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

*Вывод: В первую очередь необходимо загрузизть и изучить данные. В датасете  5 000 строк, пропусков нет. Были обнаружены дубликаты, но так как на решение задачи они не влияют, решила не обрабатывать. Признаками являются: пол, возраст и зарплата застрахованного, количество членов его семьи. Целевой признак: количество страховых выплат клиенту за последние 5 лет.*

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** 
Качество линейной регрессии не изменится

**Обоснование:** 

Возьмем формулу для весов w, подставим в неё новую матрицу и получим связь новых весов w' и старых весов w.

Представим новую матрицу признаков как произведение старой матрицы признаков X на обратимую матрицу P.
$$
X'=XP
$$

Подставим новую матрицу признаков в формулу новых весов w'.

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

Для вывода формулы связи новых и старых весов мне понадобятся следующие свойства матриц:

$$
AA^{-1} = E
$$
$$
(AB)^T = B^TA^T
$$
$$
(AB)^{-1} = B^{-1}A^{-1}
$$
$$
AB(C)=(AB)C
$$

Применим эти свойства для нашей формулы новых весов w':

$$
w' = (X^TP^T XP)^{-1} X^TP^T y
$$
$$
w' = (P^T(X^TX)P)^{-1} X^TP^T y
$$
$$
w' = (P^T)^{-1}(X^TX)^{-1}P^{-1} X^TP^T y
$$
$$
(P^T)^{-1}P^T=E
$$
$$
w'=(X^TX)^{-1}P^{-1}X^Ty
$$
$$
(X^T X)^{-1} X^T y = w
$$
$$
w'=P^{-1}w
$$

Подставим эту формулу для предсказания a'
$$
a'=X'w'=XPP^{-1}w=Xw
$$
$$
a'=a
$$
Видим что при умножении на обратимую матрицу предсказания не меняются, а значит качество линейной регрессии не меняется. Параметры линейной регрессии в исходной задаче и в преобразованной связаны таким образом $ w'=P^{-1}w $

## Алгоритм преобразования

**Алгоритм**

С помощью функции numpy.random.normal() сгенерируем случайную матрицу и затем создадим обратимую к ней. Затем эту матрицу умножим на матрицу исходных признаков. Посчитаем r_2 в полученной матрице и r_2 исходной.

**Обоснование**

В пункте 2 мы математически доказали, что предсказания на исходных данных и предсказания при умножении на обратимую матрицу одинаковы а значит и метрика r2 не должнаразличаться 

In [5]:
features = data.drop('Страховые выплаты', axis=1) 

In [6]:
matrix_random = np.random.normal(size=(4, 4))
matrix_random_inv = np.linalg.inv(matrix_random)
matrix = features.values 
matrix_1 = matrix@matrix_random_inv 
features_new = pd.DataFrame (matrix_1, columns = features.columns)
features_new 

,Пол,Возраст,Зарплата,Члены семьи
0,45582.783855,-24237.111645,108699.454891,330393.589948
1,34987.194334,-18608.899459,83382.247443,253404.720606
2,19350.796415,-10293.811533,46105.531775,140108.267300
3,38262.100999,-20339.249405,91287.514343,277506.980328
4,24015.047182,-12771.710959,57246.022166,173982.288683
...,...,...,...,...
4995,32801.852862,-17440.411063,78225.285930,237772.894281
4996,48113.316107,-25579.378978,114766.382833,348859.742771
4997,31118.483615,-16542.842531,74233.419340,225656.527197
4998,30030.476513,-15964.841019,71627.169053,217726.523712


In [7]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

## Проверка алгоритма

In [8]:
model = LinearRegression() 
model.fit(features, target)
predictions = model.predict(features)
print('Метрика R2 на исходных данных:', r2_score(target, predictions))

Метрика R2 на исходных данных: 0.42494550286668


In [9]:
model = LinearRegression() 
model.fit(features_new, target)
predictions = model.predict(features_new)
print('Метрика R2 на преобразованных данных:', r2_score(target, predictions))

Метрика R2 на преобразованных данных: 0.424945502852491


**ВЫВОД:** После математического преобразования данных видим что качество линейной регрессии не отличается после до и после преобразований, то есть метрики R2 равны.

<div class="alert alert-info">
ВЫВОД: 
Основная задача стоит перед нами это - защитить данные клиентов страховой компании «Хоть потоп». Для этого требовалось разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Требовалось  защитить данные таким образом, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.
    Изначально я изучила данные, в датасете 5 000 строк, по каждому страхователю известна следующая информация:пол, возраст, зарплата, количество членов семьи (при построении модели -это признаки) и количество страховых выплат за последние 5 лет (целевой признак).
    Затем я преобразовала данные: сгенерировала случайную матрицу, такой же размерностью, как исходная. Далее проверила эту матрицу на обратимость функцией np.linalg.inv. Умножила эту матрицу на исходную, результатом получиласт новая матрица с новыми признаками. Далее применив метрику R2 я проверила, что качество линейной регрессии из sklearn не отличается до и после преобразования. А значит, поставленная цель выполнена.